In [75]:
import pandas as pd
from artifacts.ai_pc import main_proc_change
from config.params import ProcChangeConfig
import csv

In [71]:
df = pd.read_csv('AAM_MEXICO_DATA_ACCELERATION_EMA2252M10022.csv')

In [72]:
df['READ_TIME'] = pd.to_datetime(df['READ_TIME'])
batch_size = 8  
df['batch'] = (df['READ_TIME'] - df['READ_TIME'].iloc[0]).dt.total_seconds() // (batch_size * 3600)

grouped = df.groupby('batch')

batch_results = []
for _, group in grouped:
    batch_start_hour = group['READ_TIME'].iloc[0].strftime('%Y%m%d%H')  # Format startHour as needed
    
    batch_fetch_result = {
        "data": {
            "moldId": int(group['ID'].unique()[0]),
            "counterId": str(group['COUNTER_ID'].unique()[0]),
            "startHour": batch_start_hour,
            "contractedCycleTime": 360,
            "accelerations": group['ACCELERATIONS'].tolist(),
            "dataAccId": group['ID'].tolist(),
            "measurementDate": group['MEASUREMENT_DATE'].astype(str).tolist(),
            "procChanged": [None] * len(group),
            "similarityMetric": [None] * len(group),
            "similarityMetricHr": [None] * len(group),
        }
    }
    batch_results.append(batch_fetch_result)


In [73]:
result=main_proc_change(batch_fetch_result,
                THRESHOLD_SIM_METRIC=ProcChangeConfig.THRESHOLD_SIM_METRIC,
                N_PREV_RECS=ProcChangeConfig.N_PREV_RECS,
                FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
                BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
                BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION,

                )

In [74]:
print(result)

{'moldId': 43916, 'counterId': 'EMA2252M10022', 'dataAccId': [43963, 43964, 43965, 43966, 43967, 43968, 43969, 43970, 43971, 43972, 43973, 44024, 44025, 44026, 44027, 44028, 44029, 44030, 44037, 44038, 44039, 44040, 44041], 'similarityMetric': [0.897085, 0.786796, 0.897085, 0.897085, 0.892607, 0.755929, 0.893237, 0.893237, 0.782794, 0.944911, 0.842701, 0.699854, 0.808736, 0.884538, 0.989071, 0.790569, 0.866025, 0.891133, 0.971825, 0.959645, 0.723627, 0.885422, 0.923186], 'similarityMetricHr': [0.836039, 0.836039, 0.836039, 0.836039, 0.836039, 0.836039, 0.836039, 0.836039, 0.928423, 0.928423, 0.928423, 0.928423, 0.928423, 0.928423, 0.928423, 0.901242, 0.901242, 0.901242, 0.901242, 0.901242, 0.901242, 0.901242, 0.963925], 'procChanged': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [76]:
csv_file_path = 'result.csv'

with open(csv_file_path, 'w', newline='') as csvfile:
    # Define the CSV writer
    csv_writer = csv.writer(csvfile)
    
    # Write the header row
    csv_writer.writerow(result.keys())
    
    # Transpose the values and write as rows
    num_rows = len(result["dataAccId"])  # Number of rows in the CSV
    for i in range(num_rows):
        row_values = [result[key][i] if isinstance(result[key], list) else result[key] for key in result.keys()]
        csv_writer.writerow(row_values)

print(f"Result saved to {csv_file_path}")

Result saved to result.csv
